# Intelligent Query Processing: Batch Mode on Rowstore Example

## Step 1: Enable IQP
Change the database compatibility mode to 150 to enable iQP for all queries for the WideWorldImportersDW database

In [1]:
-- Step 1: Enable IQP
--
USE [master]
GO
-- First put the database in compatibility mode of SQL Server 2019 to enable IQP
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 150
GO

Commands completed successfully.

Total execution time: 00:00:00.0035625

Commands completed successfully.

Total execution time: 00:00:00.0163720

## Step 2: Warm the cache
To make any query comparison fair let load up the pages for the QueryHhistorExtended table into cache so each query can access data from the buffer pool. Note the table has around 30M rows.


In [1]:
-- Step 2: Warm the cache
--
USE [WideWorldImportersDW]
GO
-- Warm the cache to give each query the same fair chance
SELECT COUNT(*) FROM Fact.OrderHistoryExtended
GO

Commands completed successfully.

Total execution time: 00:00:00.0020707

(1 row affected)

Total execution time: 00:00:00.3560125

(No column name)
29620736


## Step 3: Disable batch mode for rowstore with ALTER DATABASE
Scope the database to now allow batch mode for row store indexes

In [2]:
-- Step 3: Disable batch mode for rowstore with ALTER DATABASE
-- Scope the database to not allow batch mode for row store indexes
ALTER DATABASE SCOPED CONFIGURATION SET BATCH_MODE_ON_ROWSTORE = OFF
GO

Commands completed successfully.

Total execution time: 00:00:00.0060654

## Step 4: Run a query that on a multi-CPU machine should use a parallel plan and scan
Here is a typical "analytical" query use aggegration and GROUP BY against a large set of rows. There is  WHERE clause but it is a "range" query meaning a larger number of rows qualify. Notice the SET STATISTICS XML option turned on for the query to get the actual execution plan after it completes. This query should run in about 5 seconds.

In [3]:
-- Step 4: Run a query that on a multi-CPU machine should use a parallel plan and scan
-- Run with STATISTICS XML so the plan can be examined after query execution. On my computer
-- this takes about 5 seconds.
SET STATISTICS XML ON
GO
SELECT [Tax Rate], [Lineage Key], [Salesperson Key], SUM(Quantity) AS SUM_QTY, 
SUM([Unit Price]) AS SUM_BASE_PRICE, COUNT(*) AS COUNT_ORDER
FROM Fact.OrderHistoryExtended
WHERE [Order Date Key]<=DATEADD(dd, -73, '2015-11-13')
GROUP BY [Tax Rate], [Lineage Key], [Salesperson Key]
ORDER BY [Tax Rate], [Lineage Key], [Salesperson Key]
GO
SET STATISTICS XML OFF
GO

Commands completed successfully.

Total execution time: 00:00:00.0019052

(100 rows affected)

(1 row affected)

Total execution time: 00:00:04.9871230

Commands completed successfully.

Total execution time: 00:00:00.0010252

Tax Rate,Lineage Key,Salesperson Key,SUM_QTY,SUM_BASE_PRICE,COUNT_ORDER
15.000,9,4,433280,243367.68,9216
15.000,9,6,836480,543988.48,18432
15.000,9,7,724480,846118.40,24576
15.000,9,8,862080,1028780.80,22144
15.000,9,9,1101056,924917.76,28928
15.000,9,11,2279296,3886913.28,62848
15.000,9,12,3262592,3852878.08,85888
15.000,9,15,5484800,6343098.88,144256
15.000,9,19,91831040,107590420.48,2310528
15.000,9,21,1240960,2060285.44,37632


Microsoft SQL Server 2005 XML Showplan


## Step 5: Now allow batch mode for rowstore to see the difference
Scope the database to now allow batch mode for row store indexes

In [4]:
-- Step 5: Now allow batch mode for rowstore to see the difference
-- Scope the database to now allow batch mode for row store indexes
ALTER DATABASE SCOPED CONFIGURATION SET BATCH_MODE_ON_ROWSTORE = ON;
GO

Commands completed successfully.

Total execution time: 00:00:00.0072059

## Step 6: Run the same query with batch mode for store enabled
Now let's run it again and see it finish in 1 second or less

In [6]:
-- Step 6: Run the same query with batch mode for store enabled
-- Now let's run it again and see it finish in 1 second or less
SET STATISTICS XML ON
GO
SELECT [Tax Rate], [Lineage Key], [Salesperson Key], SUM(Quantity) AS SUM_QTY, 
SUM([Unit Price]) AS SUM_BASE_PRICE, COUNT(*) AS COUNT_ORDER
FROM Fact.OrderHistoryExtended
WHERE [Order Date Key]<=DATEADD(dd, -73, '2015-11-13')
GROUP BY [Tax Rate], [Lineage Key], [Salesperson Key]
ORDER BY [Tax Rate], [Lineage Key], [Salesperson Key]
GO
SET STATISTICS XML OFF
GO

Commands completed successfully.

Total execution time: 00:00:00.0011114

(100 rows affected)

(1 row affected)

Total execution time: 00:00:01.9803112

Commands completed successfully.

Total execution time: 00:00:00.0010645

Tax Rate,Lineage Key,Salesperson Key,SUM_QTY,SUM_BASE_PRICE,COUNT_ORDER
15.000,9,4,433280,243367.68,9216
15.000,9,6,836480,543988.48,18432
15.000,9,7,724480,846118.40,24576
15.000,9,8,862080,1028780.80,22144
15.000,9,9,1101056,924917.76,28928
15.000,9,11,2279296,3886913.28,62848
15.000,9,12,3262592,3852878.08,85888
15.000,9,15,5484800,6343098.88,144256
15.000,9,19,91831040,107590420.48,2310528
15.000,9,21,1240960,2060285.44,37632


Microsoft SQL Server 2005 XML Showplan
